In [55]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
import statsmodels.api as sm
from scipy import stats
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics

In [56]:
crime = pd.read_csv("updated_crime.csv")
equity = pd.read_csv("/Users/hannahjiang/Desktop/equity.csv")
census = pd.read_csv("/Users/hannahjiang/Desktop/Community Reporting Areas.csv")
neighborhood = pd.read_csv("/Users/hannahjiang/Desktop/neighborhooddata.csv")

In [57]:
neighborhood["GEOID10"] = neighborhood["GEOID10"].map(lambda x: str(x)[:-4]).astype(int)

In [58]:
crime["GEOID10"] = crime["GEOID10"].map(lambda x: str(x)[:-4]).astype(int)

equity = equity[equity['GEOID10'].isin(crime["GEOID10"].to_list())]

get_neighborhood = neighborhood.set_index("CRA_NAME")
neighborhood_dict = get_neighborhood.to_dict()["GEOID10"]

census.replace({"Community Reporting Area Names":neighborhood_dict}, inplace = True)
census.rename(columns = {"Community Reporting Area Names": "GEOID10"}, inplace = True)
#census["GEOID10"] = census["GEOID10"].map(lambda x: str(x)[:-4]).astype(int)

geo = pd.DataFrame(crime["GEOID10"].value_counts())
geo.reset_index(inplace=True)
geo.columns=['GEOID10','Count']
crime_data = geo.merge(census)
crime_data = crime_data.merge(equity)
crime_data["Total Population"] = crime_data["Total Population"].str.replace(',', '').astype(float)
crime_data["Crime Rate"] = (crime_data["Count"]/crime_data["Total Population"]).astype(float)

In [59]:
chosen_columns = ["Crime Rate", 'Average Household Size',
 'Percent of Population under 18 years', 'PCT_POP_INC_UNDER_200_POVERTY', 'GEOID10']

crime_data= crime_data[chosen_columns]
crime_data = crime_data.loc[crime_data["GEOID10"] != 53033008100]

In [60]:
crime_data.head(3)

,Crime Rate,Average Household Size,Percent of Population under 18 years,PCT_POP_INC_UNDER_200_POVERTY,GEOID10
1,2.161681,1.47,4.2,0.287987,53033008400
2,3.077618,1.99,15.1,0.417321,53033001200
3,0.988396,1.87,10.8,0.195256,53033005900


In [61]:
#We're going to predict the crime rate of these locations based on the Average Household size,
#percent of population under 18 years, and percent of population under 200% poverty

show_location = neighborhood.copy()
show_location = show_location[["GEOID10", "CRA_NAME"]]
show_location = show_location.loc[show_location["GEOID10"].isin([53033011300,53033011500])]
crime_data.merge(show_location.drop_duplicates(keep="first"))

,Crime Rate,Average Household Size,Percent of Population under 18 years,PCT_POP_INC_UNDER_200_POVERTY,GEOID10,CRA_NAME
0,1.012112,2.28,19.0,0.131413,53033011500,Roxhill/Westwood
1,1.385172,2.33,22.0,0.318208,53033011300,Highland Park


In [62]:
model_crime = crime_data.copy()
model_crime['Log Percent of Population under 18 years'] = np.log(model_crime["Percent of Population under 18 years"])
model_crime["LOG_PCT_POP_INC_UNDER_200_POVERTY"] = np.log(model_crime["PCT_POP_INC_UNDER_200_POVERTY"])
model_crime["Log Crime Rate"] = np.log(model_crime["Crime Rate"])
model_crime = model_crime.drop(["Percent of Population under 18 years","PCT_POP_INC_UNDER_200_POVERTY","Crime Rate"],axis = 1)

In [63]:
eliminate_geos = model_crime.loc[~model_crime["GEOID10"].isin([53033011300,53033011500])]
X_train = eliminate_geos.drop(["Log Crime Rate", "GEOID10"], axis = 1)
y_train = eliminate_geos["Log Crime Rate"]

only_geos = model_crime.loc[model_crime["GEOID10"].isin([53033011300,53033011500])]
X_test = only_geos.drop(["Log Crime Rate", "GEOID10"],axis = 1)
y_test = only_geos["Log Crime Rate"]

In [64]:
X = model_crime.drop(["GEOID10","Log Crime Rate"],axis = 1)
y = model_crime["Log Crime Rate"]

In [65]:
y.shape

(38,)

In [66]:
X2 = sm.add_constant(X)
est = sm.OLS(y,X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:         Log Crime Rate   R-squared:                       0.365
Model:                            OLS   Adj. R-squared:                  0.309
Method:                 Least Squares   F-statistic:                     6.505
Date:                Mon, 03 Aug 2020   Prob (F-statistic):            0.00134
Time:                        15:10:56   Log-Likelihood:                -39.116
No. Observations:                  38   AIC:                             86.23
Df Residuals:                      34   BIC:                             92.78
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

In [67]:
from sklearn.metrics import mean_squared_error

lm = linear_model.LinearRegression()
model = lm.fit(X_train, y_train)
predictions = lm.predict(X_test)

print ("Error:", mean_squared_error(np.abs(predictions),y_test))

Error: 0.10936637750569064
